In [14]:
import numpy as np
data = np.load("exercise1.npz")
XA1 = data['XA1']
yA1 = data['yA1']
XA2 = data['XA2']
yA2 = data['yA2']

XB1 = data['XB1']
yB1 = data['yB1']
XB2 = data['XB2']
yB2 = data['yB2']
data.close()

In [55]:
from sklearn import linear_model
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score


def evaluate_performance(X,y,cv_):
    reg = linear_model.LinearRegression()
    cv_results = cross_validate(reg, XA1, yA1, cv=cv_, scoring=('r2','neg_mean_squared_error'))
    sorted(cv_results.keys())
    #print(cv_results)
    ['fit_time', 'score_time', 'test_score']
    test_neg_mean_squared_error = np.mean(cv_results['test_neg_mean_squared_error'])
    test_r2 = np.mean(cv_results['test_r2'])
    return test_neg_mean_squared_error, rest_r2



In [67]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np

def random_forest_feature_selection(X, y, nr_ests = 10):
    rf = RandomForestRegressor(n_estimators = nr_ests, max_depth = 10, min_samples_leaf = 5)
    rf.fit(X, y)
    feature_importance = rf.feature_importances_
    indicies = sorted(range(len(feature_importance)), key=lambda k: feature_importance[k])
    selected_features = indicies[0:5]
    return selected_features